In [52]:
import pandas as pd
import numpy as np
import re
import string
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import json

In [51]:
entries = dict()
total_files = 1
token_limit = 50
filename = 'hadoop/hadoop'


for i in range(total_files):
    with open(filename + '-diffs-' + str(i) + '.txt') as f:
        count = 0
        is_rev = False
        is_msg = False
        is_diff = False
        msg_tokens = list()
        diff_tokens = list()
        
        revision = ''

        for line in f:
            #if count > 5:
             #   break

            if line == '\n':
                continue
                
            if 'revi####commit##gen####vt####se###spring####2018' in line:
                
                if is_diff == True:
                    entries[count] = (revision, len(msg_tokens), len(diff_tokens))
                    #print(revision, len(msg_tokens), len(diff_tokens))

                    msg_tokens = list()
                    diff_tokens = list()
                    revision = ''
                    
                    count += 1
                
                is_rev = True
                is_msg = False
                is_diff = False

                continue
            
            if 'comm####commit##gen####vt####se###spring####2018' in line:
                is_rev = False
                is_msg = True
                is_diff = False
                continue

            if 'diff####commit##gen####vt####se###spring####2018' in line:
                is_rev = False
                is_msg = False
                is_diff = True
                continue
                

            if is_rev == True:
                revision = line.strip()
            elif is_msg == True:
                msg_tokens.extend(re.findall(r"[^,.:;' ]+|[,.:;']", line.strip()))
            else:
                diff_tokens.extend(re.findall(r"[^,.:;' ]+|[,.:;']", line.strip()))

with open(filename + '-tokens.txt', 'w') as file:
    file.write(json.dumps(entries)) # use `json.loads` to do the reverse

4997

In [59]:
with open(filename + '-tokens.txt') as json_data:
    d = json.load(json_data)
    print(d['1'])

[u'cab0a4bf543f58600ae8499f5b219a452f89c827', 5, 193640]


In [ ]:
x = list()
y = list()
z = list()
for key, value in entries.iteritems():
    #if key > 10:
       # break
    if value[0] < 1 or value[0] > 500 or value[1] < 1 or value[1] > 1000:
        continue
    x.append(key)
    y.append(value[0])
    z.append(value[1])

In [ ]:
len(y)

In [ ]:
# red dashes, blue squares and green triangles
#plt.plot(x, y, 'bs', x, z, 'g^')

rcParams['figure.figsize'] = 10,10
plt.figure()
plt.boxplot(y, 0, 'gD', 0)
plt.show()

In [ ]:
total = 0
for item in z:
    total += item
total/ len(z)

In [ ]:
# red dashes, blue squares and green triangles
#plt.plot(x, y, 'bs', x, z, 'g^')
rcParams['figure.figsize'] = 10,10
plt.figure()
plt.boxplot(z, 0, 'gD', 0)
plt.show()

In [15]:
if '####commit##gen####vt####se###spring####2018' in line:
            if is_msg == False:
                print(msg_tokens)
                print(diff_tokens)
                msg_tokens = list()
                diff_tokens = list()
                count += 1

            is_msg ^= is_msg
            line = diffs.readline()
            continue
        
        #print(line)
        if is_msg == True:
            msg_tokens.extend(re.findall(r"[^,.:;' ]+|[,.:;']", line.strip()))
        else:
            diff_tokens.extend(re.findall(r"[^,.:;' ]+|[,.:;']", line.strip()))       
            
        line = diffs.readline()

['I', 'want', 'that', 'one', ',', 'it', 'is', 'great', '.']

In [20]:
data = pd.read_csv('spring-framework/spring-framework.txt', sep='\t', header=None)
data.columns = ["RevNo", "PrevRev", "Rev", "Message", "Generated"]

#remove empty string
data = data[data['Generated'] != '']
data = data[data['Message'] != '']

#remove NaN values
data = data[pd.notnull(data['Generated'])]
data = data[pd.notnull(data['Message'])]

#Remove merge commit and 'discarded' generated message
data.loc[:,'Message'] = data['Message'].apply(lambda x: '' if x.startswith("Merge") else x)
data.loc[:,'Generated'] = data['Generated'].apply(lambda x: '' if x.startswith("Discarded") else x)

#remove special characters
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))
#data.loc[:,'Message'] = data['Message'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x))

#remove digits
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: x.translate(None, string.digits))
#data.loc[:,'Message'] = data['Message'].apply(lambda x: x.translate(None, string.digits))

#split camel case words
#data.loc[:,'Message'] = data['Message'].apply(lambda x: " ".join(m.group(0) for m in re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', x)))
#data.loc[:,'Generated'] = data['Generated'].apply(lambda x: " ".join(m.group(0) for m in re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', x)))

#remove small commit messages and long generated messages
data.loc[:,'Message'] = data['Message'].apply(lambda x: '' if len(x) < 15 else x)
data.loc[:,'Generated'] = data['Generated'].apply(lambda x: '' if len(x) > 400 else x)

#again check empty and NaN values
data = data[pd.notnull(data['Generated'])]
data = data[data['Generated'] != '']
data = data[pd.notnull(data['Message'])]
data = data[data['Message'] != '']

#make two new columns 
#data.loc[:,'Ori'] = data['Message'].apply(lambda x: x.lower())
#data.loc[:,'Gen'] = data['Generated'].apply(lambda x: x.lower())

#make tokens of the newly generated columns and store in two addtional columns
data.loc[:,'OriTok'] = data['Message'].apply(lambda x: [x.strip() for x in x.split(" ")])
data.loc[:,'GenTok'] = data['Generated'].apply(lambda x: [x.strip() for x in x.split(" ")])

In [21]:
len(data)

4142

In [16]:
oriVocab = set()
genVocab = set()
for index, row in data.iterrows():
    oriVocab.update(row['OriTok'])
    genVocab.update(row['GenTok'])

In [17]:
file1 = file('spring-framework/vocab.ori', 'w')
file2 = file('spring-framework/vocab.gen', 'w')

for item in list(oriVocab):
    if len(item) <= 2:
        continue
    file1.write(item)
    file1.write('\n')
    
for item in list(genVocab):
    if len(item) <= 2:
        continue
    file2.write(item)
    file2.write('\n')

file1.close()
file2.close()

In [22]:
total = len(data)
n = int(total * 0.8)
m = total - n
train_data = data.head(n)
test_data = data.tail(m)

In [23]:
len(train_data)

3313

In [24]:
len(test_data)

829

In [25]:
train_data['Message'].to_csv('spring-framework/train.ori', sep='\n', index=False, header=False)
train_data['Generated'].to_csv('spring-framework/train.gen', sep='\n', index=False, header=False)

In [26]:
test_data['Message'].to_csv('spring-framework/test.ori', sep='\n', index=False, header=False)
test_data['Generated'].to_csv('spring-framework/test.gen', sep='\n', index=False, header=False)

test_data['Message'].to_csv('spring-framework/validation.ori', sep='\n', index=False, header=False)
test_data['Generated'].to_csv('spring-framework/validation.gen', sep='\n', index=False, header=False)